# Golden Solution (All tests should pass)


In [1]:
import numpy as np
from scipy.linalg import solve_banded

def analytic_solution_sine_mode(x: np.ndarray, t: float, alpha: float, L: float) -> np.ndarray:
    return np.sin(np.pi * x / L) * np.exp(-alpha * (np.pi / L)**2 * t)

In [2]:
def crank_nicolson_heat(u0, alpha, L, T, nx, nt):
    """Crank–Nicolson solver for u_t = alpha u_xx on [0, L] with homogeneous Dirichlet boundaries."""
    if nx < 3:
        raise ValueError("nx must be >= 3 (including endpoints).")
    if nt < 1:
        raise ValueError("nt must be >= 1.")
    if L <= 0 or T < 0:
        raise ValueError("L must be > 0 and T must be >= 0.")
    if alpha <= 0:
        raise ValueError("alpha must be > 0.")
    u0 = np.asarray(u0, dtype=float)
    if u0.shape != (nx,):
        raise ValueError(f"u0 must have shape ({nx},).")

    x = np.linspace(0.0, L, nx)
    dx = x[1] - x[0]
    u = u0.copy()
    u[0] = 0.0
    u[-1] = 0.0

    if T == 0.0:
        return x, u

    dt = T / nt
    r = alpha * dt / (dx * dx)
    n = nx - 2

    main_L = (1.0 + r) * np.ones(n)
    off_L = (-r / 2.0) * np.ones(n - 1)

    main_R = (1.0 - r) * np.ones(n)
    off_R = (r / 2.0) * np.ones(n - 1)

    ab = np.zeros((3, n))
    ab[0, 1:] = off_L
    ab[1, :] = main_L
    ab[2, :-1] = off_L

    for _ in range(nt):
        u_in = u[1:-1]
        b = main_R * u_in
        b[:-1] += off_R * u_in[1:]
        b[1:] += off_R * u_in[:-1]
        u[1:-1] = solve_banded((1, 1), ab, b)
        u[0] = 0.0
        u[-1] = 0.0

    return x, u


def estimate_alpha_grid_search(measurements, x_meas, times, L, nx, nt, alpha_grid):
    """Estimate alpha from u(x_meas, t) measurements of the first sine mode; snaps to nearest alpha_grid value."""
    measurements = np.asarray(measurements, dtype=float).ravel()
    times = np.asarray(times, dtype=float).ravel()
    alpha_grid = np.asarray(alpha_grid, dtype=float).ravel()

    if measurements.ndim != 1 or times.ndim != 1:
        raise ValueError("measurements and times must be 1D arrays.")
    if measurements.size != times.size:
        raise ValueError("measurements and times must have same length.")
    if times.size < 2:
        raise ValueError("Need at least 2 time points.")
    if np.any(np.diff(times) < 0):
        raise ValueError("times must be non-decreasing.")
    if not (0.0 <= float(x_meas) <= float(L)):
        raise ValueError("x_meas must be within [0, L].")
    if np.any(alpha_grid <= 0):
        raise ValueError("alpha_grid must be positive.")

    mask = (times > 0) & (np.abs(measurements) > 1e-15)
    t = times[mask]
    y = measurements[mask]
    if t.size < 2:
        raise ValueError("Not enough valid time points to estimate alpha.")

    z = np.log(np.abs(y))
    t_mean = float(np.mean(t))
    z_mean = float(np.mean(z))
    denom = float(np.sum((t - t_mean) ** 2))
    if denom <= 0.0:
        raise ValueError("Degenerate time array.")

    slope = float(np.sum((t - t_mean) * (z - z_mean)) / denom)
    k = (np.pi / float(L)) ** 2
    alpha_hat = -slope / k

    return float(alpha_grid[np.argmin(np.abs(alpha_grid - alpha_hat))])


def half_life_first_mode(alpha, L):
    """Half-life of the first sine mode amplitude for the 1D heat equation."""
    alpha = float(alpha)
    L = float(L)
    if alpha <= 0:
        raise ValueError("alpha must be > 0.")
    if L <= 0:
        raise ValueError("L must be > 0.")
    return float(np.log(2.0) / (alpha * (np.pi / L) ** 2))


In [4]:
import unittest
import numpy as np

class TestSubtaskA(unittest.TestCase):
    def test_output_exists_and_shape(self):
        L = 1.0
        nx = 51
        x = np.linspace(0, L, nx)
        u0 = np.sin(np.pi * x / L)
        u0[0] = 0.0
        u0[-1] = 0.0
        alpha = 1.25e-4
        T = 250.0
        nt = 200
        x_out, uT = crank_nicolson_heat(u0=u0, alpha=alpha, L=L, T=T, nx=nx, nt=nt)
        self.assertEqual(x_out.shape, (nx,))
        self.assertEqual(uT.shape, (nx,))

    def test_correctness_against_analytic(self):
        L = 1.0
        nx = 101
        x = np.linspace(0, L, nx)
        u0 = np.sin(np.pi * x / L)
        u0[0] = 0.0
        u0[-1] = 0.0
        alpha = 1.30e-4
        T = 400.0
        nt = 400
        x_out, uT = crank_nicolson_heat(u0=u0, alpha=alpha, L=L, T=T, nx=nx, nt=nt)
        u_exact = analytic_solution_sine_mode(x_out, T, alpha=alpha, L=L)
        interior = slice(1, -1)
        denom = np.maximum(np.abs(u_exact[interior]), 1e-12)
        rel_err = np.max(np.abs(uT[interior] - u_exact[interior]) / denom)
        self.assertLess(rel_err, 3e-3)

class TestSubtaskB(unittest.TestCase):
    def test_alpha_estimation(self):
        L = 1.0
        alpha_true = 1.40e-4
        nx = 61
        nt = 240
        times = np.linspace(0, 300.0, 31)
        x_meas = 0.37
        measurements = np.array([analytic_solution_sine_mode(np.array([x_meas]), t, alpha_true, L)[0] for t in times])
        alpha_grid = np.linspace(0.8e-4, 2.2e-4, 71)
        alpha_hat = estimate_alpha_grid_search(measurements, x_meas, times, L, nx, nt, alpha_grid)
        self.assertLess(abs(alpha_hat - alpha_true) / alpha_true, 0.05)

class TestSubtaskC(unittest.TestCase):
    def test_half_life_formula(self):
        L = 1.0
        alpha = 1.5e-4
        expected = np.log(2.0) / (alpha * (np.pi / L)**2)
        self.assertAlmostEqual(half_life_first_mode(alpha, L), expected, places=12)

    def test_half_life_validation(self):
        with self.assertRaises(ValueError):
            half_life_first_mode(-1.0, 1.0)
        with self.assertRaises(ValueError):
            half_life_first_mode(1.0, 0.0)

if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False, verbosity=2)

test_correctness_against_analytic (__main__.TestSubtaskA.test_correctness_against_analytic) ... ok
test_output_exists_and_shape (__main__.TestSubtaskA.test_output_exists_and_shape) ... ok
test_alpha_estimation (__main__.TestSubtaskB.test_alpha_estimation) ... ok
test_half_life_formula (__main__.TestSubtaskC.test_half_life_formula) ... ok
test_half_life_validation (__main__.TestSubtaskC.test_half_life_validation) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.053s

OK
